In [1]:
import csv, h5py
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
dset = h5py.File('./PersistenceDiagrams/jetPDs_Mass60-95_pT250-300_R1.25_Pix25.hdf5','r')
csv_file = open('./Statistics/t-statistic.csv','r')
totnum = len(dset['pd_birth'])

reader = csv.reader(csv_file, delimiter=' ', quotechar='|')
t = np.array([])
for row in reader:
    t = np.append(t, float(row[0]))

X = []
y = []

for i in range(totnum):
    
    tstat = t[i] 
    jet_delta_R = dset['jet_delta_R'][i]
    jet_eta = dset['jet_eta'][i]
    jet_mass = dset['jet_mass'][i]
    jet_phi = dset['jet_phi'][i]
    jet_pt = dset['jet_pt'][i]
    tau_1 = dset['tau_1'][i]
    tau_2 = dset['tau_2'][i]
    tau_21 = dset['tau_21'][i]
    tau_3 = dset['tau_3'][i]
    tau_32 = dset['tau_32'][i]
    
    X.append(np.array([tstat, jet_delta_R,jet_eta,jet_mass,jet_phi,jet_pt,tau_1,tau_2,tau_21,tau_3,tau_32]))
    #X.append(np.array([tstat, jet_mass, tau_21]))
    #X.append(np.array([jet_mass, tau_21]))
    y.append(int(dset['signal'][i]))

In [3]:
clf = RandomForestClassifier(n_estimators = 500, criterion = 'entropy')

In [4]:
clf.fit(X[:2000],y[:2000])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [5]:
print(clf.feature_importances_)

[0.15025697 0.07241742 0.05349153 0.14117368 0.05180827 0.06047246
 0.0750006  0.07501698 0.14675138 0.10404257 0.06956813]


In [6]:
output = clf.predict_proba(X)
csv_file = open('./Statistics/rf-statistic.csv','w')
csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

for j in range(len(output)):
    csv_writer.writerow([output[j][1]])

csv_file.close()